In [1]:
%load_ext autoreload
%autoreload 2

### Imports

In [2]:
import os
import time
import wandb
import torch
import random
import torchvision

import numpy as np
import pandas as pd
import torchmetrics as tm 
# import plotly.express as px
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from torch import nn
from pathlib import Path, PurePath
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW, RMSprop # optmizers
from sklearn import preprocessing 
# from warmup_scheduler import GradualWarmupScheduler
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau # Learning rate schedulers

import albumentations as A
# from albumentations.pytorch import ToTensorV2

import torch.nn.functional as F

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import Callback, LearningRateMonitor
from torchmetrics.wrappers import ClasswiseWrapper
from torchmetrics import MetricCollection
from torchmetrics.classification import MultilabelAccuracy, MultilabelPrecision, MultilabelRecall, MultilabelF1Score

import timm

In [3]:
print('timm version', timm.__version__)
print('torch version', torch.__version__)

timm version 1.0.8
torch version 2.3.1


In [4]:
wandb.login(key=os.getenv('wandb_api_key'))

wandb: Currently logged in as: rosu-lucian. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Asus\.netrc


True

In [5]:
# detect and define device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [6]:
# for reproducibility
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

### Config

In [7]:
# TODO: maybe use condition and level for classes
classes = ['SCS', 'RNFN', 'LNFN', 'LSS', 'RSS'] + ['H'] # add healthy class

# classes = ['SCS', 'RNFN', 'LNFN'] + ['H'] # add healthy class

# classes = ['LSS', 'RSS'] + ['H'] # add healthy class

# classes = ['SCSL1L2', 'SCSL2L3', 'SCSL3L4', 'SCSL4L5', 'SCSL5S1', 'RNFNL4L5',
#        'RNFNL5S1', 'RNFNL3L4', 'RNFNL1L2', 'RNFNL2L3', 'LNFNL1L2',
#        'LNFNL4L5', 'LNFNL5S1', 'LNFNL2L3', 'LNFNL3L4', 'LSSL1L2',
#        'RSSL1L2', 'LSSL2L3', 'RSSL2L3', 'LSSL3L4', 'RSSL3L4', 'LSSL4L5',
#        'RSSL4L5', 'LSSL5S1', 'RSSL5S1'] + ['H']

num_classes = len(classes)
class2id = {b: i for i, b in enumerate(classes)}

In [103]:
train_dir = Path('E:\data\RSNA2024')

class CFG:

    project = 'lstm'
    comment = 'first'

    ### model
    model_name = 'eca_nfnet_l0' # 'resnet34', 'resnet200d', 'efficientnet_b1_pruned', 'efficientnetv2_m', efficientnet_b7 

    image_size = 256
    
    ROOT_FOLDER = train_dir
    IMAGES_DIR = ROOT_FOLDER / 'train_images'
    PNG_DIR = ROOT_FOLDER / f'pngs_{image_size}'
    FILES_CSV = ROOT_FOLDER / 'train_files.csv'
    TRAIN_CSV = ROOT_FOLDER / 'train.csv'
    TRAIN_DESC_CSV = ROOT_FOLDER / 'train_series_descriptions.csv'
    COORDS_CSV = ROOT_FOLDER / 'train_label_coordinates.csv'

    # ckpt_path = Path(r"E:\data\RSNA2024\results\ckpt\eca_nfnet_l0 5e-05 10 eps all-labels\ep_03_loss_0.15231.ckpt")
    embeds_path = Path(r"E:\data\RSNA2024\embeddings")
    stacked_path = Path(r"E:\data\RSNA2024\embeddings_stacked")

    RESULTS_DIR = train_dir / 'results'
    CKPT_DIR = RESULTS_DIR / 'ckpt'

    input_dim = 64
    hidden_dim = 64
    target_size = 64

    classes = classes

    split_fraction = 0.95

    MIXUP = False

    ### training
    BATCH_SIZE = 16
    
    ### Optimizer
    N_EPOCHS = 10
    USE_SCHD = False
    WARM_EPOCHS = 3
    COS_EPOCHS = N_EPOCHS - WARM_EPOCHS

    # LEARNING_RATE = 5*1e-5 # best
    LEARNING_RATE = 5e-5
    
    weight_decay = 1e-6 # for adamw

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    ### split train and validation sets
    num_workers = 16

    random_seed = 42

CFG.N_LABELS = len(CFG.classes)

seed_torch(seed = CFG.random_seed)

In [9]:
CFG.N_LABELS 

6

### Load data

In [10]:
train_df = pd.read_csv(CFG.TRAIN_CSV)
train_desc_df = pd.read_csv(CFG.TRAIN_DESC_CSV)
coords_df = pd.read_csv(CFG.COORDS_CSV)
files_df = pd.read_csv(CFG.FILES_CSV)

train_df.shape, train_desc_df.shape, coords_df.shape, files_df.shape

((1975, 26), (6294, 3), (48692, 18), (147218, 21))

In [11]:
train_df.fillna('N', inplace=True)

In [12]:
train_df.head(2)

,study_id,SCSL1L2,SCSL2L3,SCSL3L4,SCSL4L5,SCSL5S1,LNFNL1L2,LNFNL2L3,LNFNL3L4,LNFNL4L5,...,LSSL1L2,LSSL2L3,LSSL3L4,LSSL4L5,LSSL5S1,RSSL1L2,RSSL2L3,RSSL3L4,RSSL4L5,RSSL5S1
0,4003253,N,N,N,N,N,N,N,N,M,...,N,N,N,M,N,N,N,N,N,N
1,4646740,N,N,M,S,N,N,N,N,M,...,N,N,N,S,N,N,M,M,M,N


In [13]:
le = preprocessing.LabelEncoder() 
le.fit(train_df.iloc[:, 1])

le.classes_
# foo = le.fit_transform(train_df.iloc[:,1])

array(['M', 'N', 'S'], dtype=object)

In [14]:
train_df.iloc[:,1:] = train_df.iloc[:,1:].apply(le.fit_transform)

In [15]:
train_df.head(2)

,study_id,SCSL1L2,SCSL2L3,SCSL3L4,SCSL4L5,SCSL5S1,LNFNL1L2,LNFNL2L3,LNFNL3L4,LNFNL4L5,...,LSSL1L2,LSSL2L3,LSSL3L4,LSSL4L5,LSSL5S1,RSSL1L2,RSSL2L3,RSSL3L4,RSSL4L5,RSSL5S1
0,4003253,1,1,1,1,1,1,1,1,0,...,1,1,1,0,1,1,1,1,1,1
1,4646740,1,1,0,2,1,1,1,1,0,...,1,1,1,2,1,1,0,0,0,1


In [16]:
coords_df.sample(2)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description,rows,columns,filename,patientposition,x_perc,y_perc,inst_perc
8470,765688458,1017097760,18,RSS,L2L3,113.866365,131.044168,765688458_1017097760,765688458_1017097760_18,RSSL2L3,Axial T2,256,256,E:\data\RSNA2024\pngs_256\765688458_1017097760...,HFS,0.444790,0.511891,0.425000
37793,3329250043,134630734,5,RNFN,L5S1,378.126195,537.246654,3329250043_134630734,3329250043_134630734_5,RNFNL5S1,Sagittal T1,760,640,E:\data\RSNA2024\pngs_256\3329250043_134630734...,HFS,0.590822,0.706903,0.190476


In [17]:
coords_df.condition.unique()

array(['SCS', 'RNFN', 'LNFN', 'LSS', 'RSS'], dtype=object)

In [18]:
coords_df.cl.unique()

array(['SCSL1L2', 'SCSL2L3', 'SCSL3L4', 'SCSL4L5', 'SCSL5S1', 'RNFNL4L5',
       'RNFNL5S1', 'RNFNL3L4', 'RNFNL1L2', 'RNFNL2L3', 'LNFNL1L2',
       'LNFNL4L5', 'LNFNL5S1', 'LNFNL2L3', 'LNFNL3L4', 'LSSL1L2',
       'RSSL1L2', 'LSSL2L3', 'RSSL2L3', 'LSSL3L4', 'RSSL3L4', 'LSSL4L5',
       'RSSL4L5', 'LSSL5S1', 'RSSL5S1'], dtype=object)

In [19]:
coords_df.cl.nunique()

25

In [20]:
embed_files = os.listdir(CFG.embeds_path)

len(embed_files)

147218

In [21]:
study_id = 838134337

In [22]:
selected_files = files_df[files_df.study_id == study_id]

selected_files.head(2)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,...,ss_id,instance_id,filename,series_description,cl,condition,inst_min,inst_max,inst,inst_perc
140320,838134337,1285354049,1,19,1,448,448,4.0,4.48,HFS,...,838134337_1285354049,838134337_1285354049_1,E:\data\RSNA2024\pngs_256\838134337_1285354049...,Sagittal T2/STIR,H,H,1,20,0,0.00
140321,838134337,1285354049,10,-20,10,448,448,4.0,4.48,HFS,...,838134337_1285354049,838134337_1285354049_10,E:\data\RSNA2024\pngs_256\838134337_1285354049...,Sagittal T2/STIR,H,H,1,20,9,0.45


In [23]:
# selected_files.groupby('series_description').sort(['proj'])

In [24]:
for name, group in selected_files.sort_values('proj').groupby('series_description'):
    print(name)
    print(group.image.count())

Axial T2
63
Sagittal T1
20
Sagittal T2/STIR
20


In [25]:
files = files_df[files_df.study_id == study_id].instance_id.to_list()
files = [CFG.embeds_path / f'{f}.npy' for f in files]

files[0]

WindowsPath('E:/data/RSNA2024/embeddings/838134337_1285354049_1.npy')

In [26]:
np.load(files[2])

array([0.19990681, 0.13388744, 0.081677  , 0.7449569 , 0.11782445,
       0.71174335, 0.7239998 , 0.16039702, 0.23803055, 0.7323503 ,
       0.78617513, 0.9582367 , 0.12478404, 0.7033064 , 0.5421161 ,
       0.8636893 , 0.80034846, 0.81664217, 0.45245513, 0.88537997,
       0.676709  , 0.3442399 , 0.8683454 , 0.93591374, 0.22409391,
       0.9467593 , 0.42132142, 0.6168075 , 0.2643018 , 0.14936103,
       0.6911781 , 0.8419221 , 0.40297708, 0.65192336, 0.58653873,
       0.2654458 , 0.7226292 , 0.4850632 , 0.88341177, 0.21312091,
       0.5018644 , 0.28670755, 0.5729891 , 0.67702264, 0.93779194,
       0.33582947, 0.75234455, 0.8283155 , 0.40923488, 0.28497517,
       0.27483886, 0.31631535, 0.3859773 , 0.2297201 , 0.93978065,
       0.1383495 , 0.7521539 , 0.5270227 , 0.29936162, 0.9263395 ,
       0.8865664 , 0.9513482 , 0.71094906, 0.8872427 ], dtype=float32)

In [27]:
train_df[train_df.study_id == study_id].values.flatten().tolist()[1:]

[1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1]

### Dataset

In [28]:
from dataset import rsna_lstm_dataset

In [30]:
dset = rsna_lstm_dataset(train_df, files_df, CFG.embeds_path)

print(dset.__len__())

seq, target = dset.__getitem__(1)
print(seq.shape, target.shape)
print(seq.dtype, target.dtype)

1975
torch.Size([88, 64]) torch.Size([25])
torch.float32 torch.int64


In [31]:
# seq dim: (bs, seq_len, 1, num_features)
# target dim: (N, d1)
target

tensor([1, 1, 0, 2, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0,
        1])

In [32]:
seq[0]

tensor([0.4239, 0.5589, 0.5724, 0.3233, 0.5413, 0.4105, 0.2450, 0.2933, 0.5783,
        0.3665, 0.2999, 0.3192, 0.6060, 0.4276, 0.7149, 0.6555, 0.4987, 0.2461,
        0.5538, 0.3502, 0.3036, 0.5713, 0.4331, 0.4855, 0.2895, 0.6407, 0.5820,
        0.2301, 0.5089, 0.4737, 0.4656, 0.6288, 0.4501, 0.5102, 0.2545, 0.6618,
        0.2734, 0.5166, 0.4853, 0.6639, 0.4464, 0.7453, 0.3942, 0.4599, 0.4279,
        0.5163, 0.4440, 0.4644, 0.7640, 0.5849, 0.7543, 0.5833, 0.7475, 0.7489,
        0.3839, 0.4694, 0.2550, 0.4448, 0.2281, 0.6770, 0.6206, 0.5454, 0.2657,
        0.5847])

### Data Module

In [43]:
from dataset import rsna_lstm_dataset, collate_fn_padd

In [44]:
# from torch.nn.utils.rnn import pad_sequence

# def collate_fn_padd(data):
#     tensors, targets = zip(*data)
#     features = pad_sequence(tensors, batch_first=True)
#     targets = torch.stack(targets)
#     return features, targets
    
class lstm_datamodule(pl.LightningDataModule):
    def __init__(self, train_df, val_df, files_df, cfg):
        super().__init__()
        
        self.train_df = train_df
        self.val_df = val_df
        self.files_df = files_df
        
        self.train_bs = cfg.BATCH_SIZE
        self.val_bs = cfg.BATCH_SIZE

        self.cfg = cfg
        
        self.num_workers = cfg.num_workers
        
    def train_dataloader(self):
        train_ds = rsna_lstm_dataset(self.train_df, self.files_df, self.cfg.embeds_path)
        
        train_loader = torch.utils.data.DataLoader(
            train_ds,
            batch_size=self.train_bs,
            collate_fn=collate_fn_padd,
            pin_memory=False,
            drop_last=False,
            shuffle=True,
            persistent_workers=True,
            num_workers=self.num_workers,
        )
        
        return train_loader
        
    def val_dataloader(self):
        val_ds = rsna_lstm_dataset(self.val_df, self.files_df, self.cfg.embeds_path)
        
        val_loader = torch.utils.data.DataLoader(
            val_ds,
            batch_size=self.val_bs,
            collate_fn=collate_fn_padd,
            pin_memory=False,
            drop_last=False,
            shuffle=False,
            persistent_workers=True,
            num_workers=2,
        )
        
        return val_loader

In [46]:
t_df = train_df[:-100]
# t_df = pd.concat([meta_df[:-100], ul_df[:-100]], ignore_index=True)
v_df = train_df[-100:]

CFG2 = CFG()
# CFG2 = copy.deepcopy(CFG)
CFG2.BATCH_SIZE = 8
CFG2.num_workers = 4

dm = lstm_datamodule(t_df, v_df, files_df, CFG2)

x, y = next(iter(dm.train_dataloader()))
x.shape, y.shape, x.dtype, y.dtype

(torch.Size([8, 93, 64]), torch.Size([8, 25]), torch.float32, torch.int64)

In [47]:
del dm

### Loss function

In [48]:
class FocalLossBCE(torch.nn.Module):
    def __init__(
            self,
            alpha: float = 0.25,
            gamma: float = 2,
            reduction: str = "mean",
            bce_weight: float = 1.0,
            focal_weight: float = 1.0,
    ):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = torch.nn.BCEWithLogitsLoss(reduction=reduction)
        self.bce_weight = bce_weight
        self.focal_weight = focal_weight

    def forward(self, logits, targets):
        focall_loss = torchvision.ops.focal_loss.sigmoid_focal_loss(
            inputs=logits,
            targets=targets,
            alpha=self.alpha,
            gamma=self.gamma,
            reduction=self.reduction,
        )
        bce_loss = self.bce(logits, targets)
        return self.bce_weight * bce_loss + self.focal_weight * focall_loss

class GeM(torch.nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = torch.nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        bs, ch, h, w = x.shape
        x = torch.nn.functional.avg_pool2d(x.clamp(min=self.eps).pow(self.p), (x.size(-2), x.size(-1))).pow(
            1.0 / self.p)
        x = x.view(bs, ch)
        return x

### Model

#### Definition

In [100]:
class LSTMClassifier(pl.LightningModule):
    def __init__(self, cfg=CFG):
        super(LSTMClassifier, self).__init__()

        self.cfg = cfg
        
        self.input_dim = cfg.input_dim
        self.hidden_dim = cfg.hidden_dim
        self.target_size = cfg.target_size

        self.criterion = torch.nn.CrossEntropyLoss(reduction='none')

        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, num_layers=1, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.target_size)
        self.classifiers = torch.nn.ModuleList([nn.Linear(self.target_size, 3) for i in range(25)])

        wrapped_metric = ClasswiseWrapper(MultilabelAccuracy(num_labels=self.cfg.N_LABELS, average='none'), labels=classes, prefix='multiacc/')
        
        metrics = MetricCollection({
            'macc': MultilabelAccuracy(num_labels=self.cfg.N_LABELS),
            'macc_none': wrapped_metric,
            'mpr': MultilabelPrecision(num_labels=self.cfg.N_LABELS),
            'mrec': MultilabelRecall(num_labels=self.cfg.N_LABELS),
            'f1': MultilabelF1Score(num_labels=self.cfg.N_LABELS)
        })

        self.train_metrics = metrics.clone(prefix='train/')
        self.valid_metrics = metrics.clone(prefix='val/')

    def forward(self, sequence):
        #  seq: (seq_len, bs, num_features)
        lstm_out, (h, c) = self.lstm(sequence)
        
        y = self.fc(h[-1])

        preds = [c(y).T for c in self.classifiers]
        preds = torch.stack(preds).T
        
        return preds

    def step(self, batch, batch_idx, mode='train'):
        x, y = batch

        preds = self(x)

        loss = self.criterion(preds, y)

        # https://discuss.pytorch.org/t/pytorchs-non-deterministic-cross-entropy-loss-and-the-problem-of-reproducibility/172180/9
        loss = loss.mean()

        if mode == 'train':
            output = self.train_metrics(preds, y)
            self.log_dict(output)
        else:
            self.valid_metrics.update(preds, y)

        self.log(f'{mode}/loss', loss, on_step=True, on_epoch=True)

        return loss

    def training_step(self, batch, batch_idx):
        loss = self.step(batch, batch_idx, mode='train')
        
        return loss
        
    def validation_step(self, batch, batch_idx):
        loss = self.step(batch, batch_idx, mode='val')
    
        return loss

    def on_train_epoch_end(self):
        self.train_metrics.reset()

    def on_validation_epoch_end(self):
        output = self.valid_metrics.compute()
        self.log_dict(output)

        self.valid_metrics.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=self.cfg.LEARNING_RATE, weight_decay=CFG.weight_decay)
        
        if self.cfg.USE_SCHD:
            scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, self.cfg.COS_EPOCHS)
            scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=self.cfg.WARM_EPOCHS, after_scheduler=scheduler_cosine)

            return [optimizer], [scheduler_warmup]
        else:
            return optimizer

#### building blocks

In [90]:
seq.shape, seq.view(1, len(seq), -1).shape, target.shape

(torch.Size([88, 64]), torch.Size([1, 88, 64]), torch.Size([25]))

In [91]:
target_size = 64

lstm = nn.LSTM(target_size, target_size, num_layers=1, batch_first=True)
fc = nn.Linear(target_size, target_size)
classifiers = [nn.Linear(target_size, 3) for i in range(25)]

lstm_out, (h, c) = lstm(torch.randn(5,88,64))
print(lstm_out.shape, h.shape, c.shape)

y = fc(h[-1])
print(y.shape)

preds = [c(y) for c in classifiers]
print(preds[0].shape)
preds = torch.stack(preds)

preds.shape

torch.Size([5, 88, 64]) torch.Size([1, 5, 64]) torch.Size([1, 5, 64])
torch.Size([5, 64])
torch.Size([5, 3])


torch.Size([25, 5, 3])

In [92]:
preds.T.shape

torch.Size([3, 5, 25])

#### Test out inputs/outputs

In [93]:
model = LSTMClassifier(CFG)

In [94]:
model.step((seq.view(1, len(seq), -1), target.view(1, len(target))), 0)

tensor(1.1038, grad_fn=<NllLoss2DBackward0>)

In [95]:
y = model(torch.randn(5,88,64))

y.shape, y.softmax(dim=0).shape

(torch.Size([5, 3, 25]), torch.Size([5, 3, 25]))

In [96]:
# y[0].softmax(dim=0)

In [97]:
y.softmax(dim=1)

tensor([[[0.3271, 0.3245, 0.3209, 0.3137, 0.3169, 0.3333, 0.3335, 0.3495,
          0.2808, 0.3891, 0.3383, 0.3145, 0.3139, 0.3272, 0.3225, 0.3030,
          0.3107, 0.3147, 0.3478, 0.3183, 0.3744, 0.2880, 0.3164, 0.3018,
          0.3393],
         [0.3019, 0.3371, 0.3630, 0.3159, 0.3218, 0.3173, 0.3573, 0.3594,
          0.3501, 0.3203, 0.3498, 0.3699, 0.3642, 0.3592, 0.3442, 0.3516,
          0.3468, 0.3281, 0.3207, 0.3701, 0.3157, 0.3521, 0.3435, 0.3611,
          0.3115],
         [0.3710, 0.3384, 0.3161, 0.3704, 0.3613, 0.3494, 0.3093, 0.2912,
          0.3691, 0.2906, 0.3118, 0.3156, 0.3219, 0.3136, 0.3334, 0.3454,
          0.3425, 0.3573, 0.3315, 0.3117, 0.3098, 0.3599, 0.3401, 0.3372,
          0.3492]],

        [[0.3225, 0.3173, 0.3134, 0.3240, 0.3301, 0.3379, 0.3205, 0.3305,
          0.3077, 0.3559, 0.3266, 0.3457, 0.2885, 0.3031, 0.3132, 0.3239,
          0.3370, 0.3166, 0.3352, 0.3168, 0.3682, 0.3075, 0.3059, 0.2769,
          0.3210],
         [0.3017, 0.3148, 0.3632, 

In [35]:
# y.softmax(dim=1).sum(dim=1)

### Split

In [58]:
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit

In [74]:
train_df.shape

(1975, 26)

In [59]:
train_df.sample(2)

,study_id,SCSL1L2,SCSL2L3,SCSL3L4,SCSL4L5,SCSL5S1,LNFNL1L2,LNFNL2L3,LNFNL3L4,LNFNL4L5,...,LSSL1L2,LSSL2L3,LSSL3L4,LSSL4L5,LSSL5S1,RSSL1L2,RSSL2L3,RSSL3L4,RSSL4L5,RSSL5S1
1454,3168755174,1,1,2,1,1,1,1,1,0,...,1,1,2,1,1,1,1,2,2,2
1762,3847067228,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [77]:
train_df.iloc[:,1:].shape

(1975, 25)

In [78]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=1-CFG.split_fraction, random_state=CFG.random_seed)
train_idx, val_idx = next(sss.split(train_df.study_id, train_df.iloc[:,1:]))

t_df = train_df.iloc[train_idx]
v_df = train_df.iloc[val_idx]

t_df.shape, v_df.shape

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

### Train

In [63]:
CFG.BATCH_SIZE, CFG.device

(16, 'cuda')

In [81]:
dm = lstm_datamodule(t_df, v_df, files_df, cfg=CFG)

len(dm.train_dataloader()), len(dm.val_dataloader())

(118, 7)

In [82]:
run_name = f'{CFG.model_name} {CFG.LEARNING_RATE} {CFG.N_EPOCHS} eps {CFG.comment}'
run_name

'eca_nfnet_l0 5e-05 10 eps first'

In [83]:
wandb_logger = WandbLogger(
    name=run_name,
    project=CFG.project,
    job_type='train',
    save_dir=CFG.RESULTS_DIR,
    # config=cfg,
)

loss_ckpt = pl.callbacks.ModelCheckpoint(
    monitor='val/loss',
    auto_insert_metric_name=False,
    dirpath=CFG.CKPT_DIR / run_name,
    filename='ep_{epoch:02d}_loss_{val/loss:.5f}',
    save_top_k=2,
    mode='min',
)

# acc_ckpt = pl.callbacks.ModelCheckpoint(
#     monitor='val/acc',
#     auto_insert_metric_name=False,
#     dirpath=CFG.CKPT_DIR / run_name,
#     filename='ep_{epoch:02d}_acc_{val/acc:.5f}',
#     save_top_k=2,
#     mode='max',
# )

lr_monitor = LearningRateMonitor(logging_interval='step')

In [84]:
trainer = pl.Trainer(
    max_epochs=CFG.N_EPOCHS,
    deterministic=True,
    accelerator=CFG.device,
    default_root_dir=CFG.RESULTS_DIR,
    gradient_clip_val=0.5, 
    # gradient_clip_algorithm="value",
    logger=wandb_logger,
    callbacks=[loss_ckpt, lr_monitor],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [101]:
model = LSTMClassifier(CFG)

#### Fit

In [102]:
trainer.fit(model, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | criterion     | CrossEntropyLoss | 0      | train
1 | lstm          | LSTM             | 33.3 K | train
2 | fc            | Linear           | 4.2 K  | train
3 | classifiers   | ModuleList       | 4.9 K  | train
4 | train_metrics | MetricCollection | 0      | train
5 | valid_metrics | MetricCollection | 0      | train
-----------------------------------------------------------
42.3 K    Trainable params
0         Non-trainable params
42.3 K    Total params
0.169     Total estimated model params size (MB)
43        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                        | 0/? [00:00<?, ?it/…

C:\ProgramData\anaconda3\envs\rsna\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MultilabelF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\ProgramData\anaconda3\envs\rsna\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MultilabelAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\ProgramData\anaconda3\envs\rsna\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric MultilabelPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
C:\ProgramData\anaconda3\envs\rsna\lib\site-packages\torchmetric

Training: |                                                                               | 0/? [00:00<?, ?it/…

Validation: |                                                                             | 0/? [00:00<?, ?it/…

MisconfigurationException: `ModelCheckpoint(monitor='val/loss')` could not find the monitored key in the returned metrics: ['lr-Adam', 'val/f1', 'val/macc', 'val/multiacc/SCS', 'val/multiacc/RNFN', 'val/multiacc/LNFN', 'val/multiacc/LSS', 'val/multiacc/RSS', 'val/multiacc/H', 'val/mpr', 'val/mrec', 'epoch', 'step']. HINT: Did you call `log('val/loss', value)` in the `LightningModule`?